<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/4-embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings

# Actividad 1: Word2vec en tensorflow

Observe el preprocesamiento que se realiza al corpus, la estructura construida con Tensorflow y el procedimiento para entrenar el modelo.


1.   Construya los word embeddings desde los pesos y sesgos del modelo entrenado
2.   Visualice los embeddings bidimensionales en un gráfico de dispersión e interprete los resultados


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

In [3]:
def preprocess(text):
  tokens = text.split(" ")
  return [token for token in tokens if len(token) > 2]

In [4]:
def get_vocabulary(corpus):
  vocabulary = []
  for text in corpus:
    vocabulary += text
  return sorted(list(set(vocabulary)))

In [5]:
sentences = list(map(preprocess, corpus))

In [6]:
vocabulary = get_vocabulary(sentences)

In [7]:
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        # Acá se toma una ventana de -WINDOWS_SIZE, WINDOWS_SIZE para generar el skip gram. Dado que las 
        # frases son cortas, se utiliza min y max para tener cuidado con los límites de la frase. 
        # Además, el +1 en el límite superior es para considerar el índice de la propia palabra en cuestión 
        # (probar qué ocurre cuando se elimina dicho +1)
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1]: 
            if neighbor != word:
                data.append((word, neighbor))

In [8]:
features = np.zeros((len(data),len(vocabulary)),dtype=np.float32)
labels = np.zeros((len(data),len(vocabulary)),dtype=np.float32)
for i,(feature,label) in enumerate(data):
  features[i,vocabulary.index(feature)] = 1
  labels[i,vocabulary.index(label)] = 1

In [9]:
class Word2Vec:
  def __init__(self, vocab_size=0, embedding_dim=2, epochs=10000):
    self.vocab_size=vocab_size
    self.embedding_dim=embedding_dim
    self.epochs=epochs
    self.optimizer = tf.optimizers.SGD(learning_rate=0.1)
  def train(self, x_train=None, y_train=None):
    self.W1 = tf.Variable(tf.random.normal([self.vocab_size, self.embedding_dim]))
    self.b1 = tf.Variable(tf.random.normal([self.embedding_dim])) #bias
 
    self.W2 = tf.Variable(tf.random.normal([self.embedding_dim, self.vocab_size]))
    self.b2 = tf.Variable(tf.random.normal([self.vocab_size]))
 
    for _ in range(self.epochs):
      with tf.GradientTape() as t:
        hidden_layer = tf.add(tf.matmul(x_train,self.W1),self.b1) 
        output_layer = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, self.W2), self.b2))
        cross_entropy_loss = tf.reduce_mean(-tf.math.reduce_sum(y_train * tf.math.log(output_layer), axis=[1]))
 
      grads = t.gradient(cross_entropy_loss, [self.W1, self.b1, self.W2, self.b2])
      self.optimizer.apply_gradients(zip(grads,[self.W1, self.b1, self.W2, self.b2]))
      if(_ % 1000 == 0):
        print(cross_entropy_loss)

In [10]:
w2v = Word2Vec(vocab_size=len(vocabulary), epochs=10000)
w2v.train(features, labels)

tf.Tensor(4.9014196, shape=(), dtype=float32)
tf.Tensor(2.0284772, shape=(), dtype=float32)
tf.Tensor(1.8472476, shape=(), dtype=float32)
tf.Tensor(1.8111458, shape=(), dtype=float32)
tf.Tensor(1.7964778, shape=(), dtype=float32)
tf.Tensor(1.7883557, shape=(), dtype=float32)
tf.Tensor(1.7831641, shape=(), dtype=float32)
tf.Tensor(1.7795072, shape=(), dtype=float32)
tf.Tensor(1.7767707, shape=(), dtype=float32)
tf.Tensor(1.774647, shape=(), dtype=float32)


In [11]:
# programa
#
#
#

## Embeddings con gensim

In [12]:
import nltk
import re
import gensim
import gensim.downloader
import logging

Descargamos el corpus Brown

In [13]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [14]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [15]:
corpus = nltk.corpus.brown.sents()

Entrenamos un modelo model sobre el corpus Brown, el cual contiene alrededor de 1.000.000 de palabras.

In [16]:
model = gensim.models.word2vec.Word2Vec(sentences = corpus)

2020-11-08 04:22:51,415 : INFO : collecting all words and their counts
2020-11-08 04:22:51,420 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-11-08 04:22:51,959 : INFO : PROGRESS: at sentence #10000, processed 219770 words, keeping 23488 word types
2020-11-08 04:22:52,412 : INFO : PROGRESS: at sentence #20000, processed 430477 words, keeping 34367 word types
2020-11-08 04:22:52,889 : INFO : PROGRESS: at sentence #30000, processed 669056 words, keeping 42365 word types
2020-11-08 04:22:53,338 : INFO : PROGRESS: at sentence #40000, processed 888291 words, keeping 49136 word types
2020-11-08 04:22:53,694 : INFO : PROGRESS: at sentence #50000, processed 1039920 words, keeping 53024 word types
2020-11-08 04:22:53,966 : INFO : collected 56057 word types from a corpus of 1161192 raw words and 57340 sentences
2020-11-08 04:22:53,967 : INFO : Loading a fresh vocabulary
2020-11-08 04:22:54,013 : INFO : effective_min_count=5 retains 15173 unique words (27% of orig

Importamos un modelo de word2vec entrenado sobre google news, este modelo fue entrenado sobre un corpus de 3.000.000.000 de palabras

In [17]:
news = gensim.downloader.load('word2vec-google-news-300')

2020-11-08 04:23:17,404 : INFO : Creating /root/gensim-data


[==================================================] 100.0% 1662.8/1662.8MB downloaded


2020-11-08 04:27:02,228 : INFO : word2vec-google-news-300 downloaded
2020-11-08 04:27:02,230 : INFO : loading projection weights from /root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2020-11-08 04:29:10,606 : INFO : loaded (3000000, 300) matrix from /root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


## Actividad 2: Exploración de los embeddings

Después de calcular los embeddings, podemos asociar un vector de números reales, de dimensiones conocidas a una palabra de nuestro vocabulario.


1.   ¿Cuántas dimensiones tiene cada vector asociado a las palabras en cada uno de los modelos?
2.   ¿Cuántos vectores hay en cada uno de los modelos?
3.   ¿Existen diferencian en el tamaño del vocabulario del modelo `model` y el modelo news?



In [18]:
# HINT
# news.wv.vectors # esta es una matriz

## Actividad 3: Similaridad y analogía

Existen un par de cualidades semánticas de las palabras, las cuales pueden ser fácilmente demostradas a través de operaciones vectoriales sobre el espacio generado por el proceso de cálculo de embeddings.

La similaridad es la métrica de cercanía que tienen 2 palabras, esta característica es fácil de representar a través de la similaridad coseno entre 2 vectores.

La analogía es la relación semántica que tienen 2 palabras, por ejemplo, la palabra "rey" y "reina" están relacionadas por el concepto de "género". Estas analogías se pueden operacionalizar en el espacio vectorial de los word embeddings como la resta de los vectores asociados a las palabras.



1.   Verifique cuáles son las palabras más cercanas a palabras que usted seleccione e interprete la correctitud de las palabras retornadas por los modelos `model` y `news`
2.   Verifique cuál de los modelos resuelve mejor la prueba de analogía: "man" es a "woman" como "king" es a "queen". Invente otra analogía y pruebe si el modelo la puede resolver.
3.   Según sus pruebas, ¿el tamaño del corpus de entrenamiento afecta el rendimiento del modelo?



In [19]:
# así obtenemos las palabras más cercanas a las palabra "woman"
model.wv.most_similar("woman")

2020-11-08 04:29:10,633 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('girl', 0.9580943584442139),
 ('boy', 0.9006985425949097),
 ('man', 0.876850962638855),
 ('child', 0.8736554384231567),
 ('young', 0.8692346811294556),
 ('fellow', 0.8601439595222473),
 ('youngster', 0.858249306678772),
 ('remark', 0.8562021255493164),
 ('artist', 0.8556320071220398),
 ('old', 0.8539116382598877)]

In [20]:
#HINT:
# news.wv.vectors[news.vocab["woman"].index] # así obtenemos el vector asociado a la palabra woman
# news.wv.similar_by_vector(vector) # así obtenemos las palabras más cercanas a un vector

## Actividad 4: Reducción de dimensionalidad y visualización

Nuestros vectores tienen más de 2 dimensiones, por lo que no es muy fácil interptretar su ubicación espacial. Utiliza un método de reducción de dimensionalidad que conozcas para reducir hacia 2 dimensiones nuestros vectores y visualiza el resultado.


*   ¿Existen agrupaciones aparentes de palabras dentro del espacio? y si es así, ¿las palabras agrupadas, son similares?



In [21]:
# prográmame
#
#
#